# Analyse exploratoire des données

In [1]:
import pandas as pd

In [3]:
df_21= pd.read_csv('data\data21.csv')
df_20= pd.read_csv('data\data20.csv')
df_19= pd.read_csv('data\data19.csv')

## Premiers constats

In [71]:
d19 = df_19.groupby('name')['name'].transform('size').value_counts()
for i in range(1,len(d19)+1):
    print("Il y a "+str(int(d19[i]/i))+" joueurs qui apparaissent "+str(i)+" fois dans la base de 2019")

Il y a 444 joueurs qui apparaissent 1 fois dans la base de 2019
Il y a 80 joueurs qui apparaissent 2 fois dans la base de 2019
Il y a 6 joueurs qui apparaissent 3 fois dans la base de 2019


In [73]:
d20 = df_20.groupby('name')['name'].transform('size').value_counts()
for i in range(1,len(d20)+1):
    print("Il y a "+str(int(d20[i]/i))+" joueurs qui apparaissent "+str(i)+" fois dans la base de 2020")

Il y a 470 joueurs qui apparaissent 1 fois dans la base de 2020
Il y a 58 joueurs qui apparaissent 2 fois dans la base de 2020
Il y a 2 joueurs qui apparaissent 3 fois dans la base de 2020


In [74]:
d21 = df_21.groupby('name')['name'].transform('size').value_counts()
for i in range(1,len(d21)+1):
    print("Il y a "+str(int(d21[i]/i))+" joueurs qui apparaissent "+str(i)+" fois dans la base de 2019")

Il y a 461 joueurs qui apparaissent 1 fois dans la base de 2019
Il y a 72 joueurs qui apparaissent 2 fois dans la base de 2019
Il y a 7 joueurs qui apparaissent 3 fois dans la base de 2019


Ceci s'explique par la possibilité de transfert de joueurs au cours de la saison : ils apparaissent ainsi dans les équipes dans lequelles ils ont joué

In [53]:
df_19[df_19.groupby('name')['name'].transform('size') > 1].sort_values(by=['name'])

,and_ones,assist_percentage,assists,assists_per_poss,block_percentage,blocking_fouls,blocks,blocks_per_poss,box_plus_minus,center_percentage,...,two_pointers,two_pointers_per_poss,two_pointers_assisted_percentage,usage_percentage,value_over_replacement_player,weight,win_shares,win_shares_per_48_minutes,name,team
558,NaN,13.7,128.0,4.6,1.3,NaN,21.0,0.7,-1.2,0,...,131.0,4.7,NaN,19.0,0.3,214,1.5,0.052,Alec Burks,CLE
90,NaN,13.7,128.0,4.6,1.3,NaN,21.0,0.7,-1.2,0,...,131.0,4.7,NaN,19.0,0.3,214,1.5,0.052,Alec Burks,SAC
424,NaN,13.7,128.0,4.6,1.3,NaN,21.0,0.7,-1.2,0,...,131.0,4.7,NaN,19.0,0.3,214,1.5,0.052,Alec Burks,UTA
212,NaN,17.1,24.0,6.3,0.9,NaN,2.0,0.5,-7.1,0,...,11.0,2.9,NaN,16.8,-0.2,213,-0.1,-0.027,Andrew Harrison,NOP
288,NaN,17.1,24.0,6.3,0.9,NaN,2.0,0.5,-7.1,0,...,11.0,2.9,NaN,16.8,-0.2,213,-0.1,-0.027,Andrew Harrison,MEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,NaN,11.1,160.0,3.7,0.7,NaN,17.0,0.4,-1.8,0,...,129.0,3.0,NaN,17.8,0.1,220,2.7,0.062,Wesley Matthews,IND
160,NaN,11.1,160.0,3.7,0.7,NaN,17.0,0.4,-1.8,0,...,129.0,3.0,NaN,17.8,0.1,220,2.7,0.062,Wesley Matthews,DAL
548,NaN,11.1,160.0,3.7,0.7,NaN,17.0,0.4,-1.8,0,...,129.0,3.0,NaN,17.8,0.1,220,2.7,0.062,Wesley Matthews,NYK
380,NaN,9.1,82.0,3.3,1.4,NaN,21.0,0.8,-2.8,0,...,55.0,2.2,NaN,11.8,-0.2,235,1.5,0.062,Wilson Chandler,PHI


**Problème** : pour ces joueurs, leurs statistiques sont les mêmes quelque soit l'équipe pour laquelle ils jouent, et non pas leurs statistiques liées uniquement à leurs matchs joués pour chaque équipe
Selon ce que l'on va faire par la suite, nous déciderons de garder ou non l'une de ces les lignes similaires (qui ne diffèrent que par la colonne 'team')

## Stats des sur les variables qui nous intéressent

### Variable 'salary'

In [79]:
print("Il y a "+str(df_19['salary'].isnull().sum())+" salaires inconnus dans la base de 2019")
print("Il y a "+str(df_20['salary'].isnull().sum())+" salaires inconnus dans la base de 2020")
print("Il y a "+str(df_21['salary'].isnull().sum())+" salaires inconnus dans la base de 2021")

Il y a 49 salaires inconnus dans la base de 2019
Il y a 51 salaires inconnus dans la base de 2020
Il y a 60 salaires inconnus dans la base de 2021


In [16]:
df_19nd = df_19.groupby('name', group_keys=False).apply(lambda df: df.sample(1))
# nd pour "non duplicates"

In [21]:
df_20nd = df_20.groupby('name', group_keys=False).apply(lambda df: df.sample(1))
df_21nd = df_21.groupby('name', group_keys=False).apply(lambda df: df.sample(1))

In [17]:
print(len(df_19nd),len(df_19))

530 622


In [30]:
df_19nd['salary'].sort_values(ascending=False)

609    37457154.0
51     35654150.0
474    35654150.0
202    35654150.0
520    32088932.0
          ...    
604           NaN
190           NaN
301           NaN
525           NaN
66            NaN
Name: salary, Length: 530, dtype: float64

In [31]:
df_20nd['salary'].sort_values(ascending=False)

575    40231758.0
446    38506482.0
197    38506482.0
188    38199000.0
45     37436858.0
          ...    
230           NaN
292           NaN
465           NaN
277           NaN
200           NaN
Name: salary, Length: 530, dtype: float64

In [32]:
df_21nd['salary'].sort_values(ascending=False)

611    43006362.0
264    41358814.0
241    41358814.0
204    41254920.0
184    41254920.0
          ...    
302           NaN
423           NaN
228           NaN
141           NaN
220           NaN
Name: salary, Length: 540, dtype: float64

In [37]:
df_19nd.groupby('team')['salary'].hist()

team
ATL      47371.0
BOS     838464.0
BRK       4737.0
CHI      88531.0
CHO     988464.0
CLE      76236.0
DAL     838464.0
DEN     311070.0
DET     838464.0
GSW     486892.0
HOU     137376.0
IND     449794.0
LAC    1349383.0
LAL      52108.0
MEM      47371.0
MIA       9474.0
MIL     456733.0
MIN     203695.0
NOP     194220.0
NYK      42634.0
OKC     151587.0
ORL     838464.0
PHI      92857.0
PHO     198580.0
POR     838464.0
SAC      94742.0
SAS      59820.0
TOR     457418.0
UTA    1378242.0
WAS      17092.0
Name: salary, dtype: float64